In [35]:
import pandas as pd
import geopandas as gpd
from datetime import date, datetime
import os
import re

pd.options.display.max_columns = None

In [36]:
fires = pd.DataFrame()
pathname = '../data'

for filename in os.listdir(pathname):
    if os.path.isfile(os.path.join(pathname, filename)) and 'Confirmed' in filename:
        if os.path.splitext(filename)[1] == '.csv':
            fires = pd.concat([fires, pd.read_csv(os.path.join(pathname, filename))], ignore_index=True)

fires = fires.drop('OBJECTID', axis=1).drop_duplicates()

In [37]:
def strip_time(entry):
    m = re.match('(\d{4}/\d{2}/\d{2})', entry)
    return m.group(1)

In [38]:
fires['confirmed_fire'] = True
fires['alarm_datetime'] = fires.apply(lambda row: strip_time(row.alm_date) + ' ' + row.alm_time, axis=1)
fires['clear_datetime'] = fires.apply(lambda row: strip_time(row.clr_date) + ' ' + row.clr_time, axis=1)
fires['alarm_datetime'] = pd.to_datetime(fires['alarm_datetime'])
fires['clear_datetime'] = pd.to_datetime(fires['clear_datetime'])

fires['alm_date'] = fires['alm_date'].apply(strip_time)
fires['clr_date'] = fires['clr_date'].apply(strip_time)

fires = fires.drop(['alm_time', 'clr_time', 'X', 'Y', 'LastUpdateDate', 'StartDate', 'EndDate'], axis=1)
fires = fires.sort_values('inci_no')



fires.head()

,FID,inci_no,descript,alm_date,station,number_,street,st_type,st_suffix,addr_2,apt_room,xst_prefix,xstreet,xst_type,xst_suffix,latitude,longitude,inci_type,clr_date,alarms,complete,number,confirmed_fire,alarm_datetime,clear_datetime
38236,NaN,0026882,Dumpster or other outside trash receptacle fir...,2016/08/02,21,NaN,38,ST,E,,,,,,,0.0,0.0,154,2016/08/02,1,1,2716,True,2016-08-02 05:44:50,2016-08-02 06:20:14
38237,NaN,0026886,Dumpster or other outside trash receptacle fir...,2016/08/02,06,NaN,Franklin,AVE,W,,,S,Hennepin,AVE,,0.0,0.0,154,2016/08/02,1,1,,True,2016-08-02 06:16:10,2016-08-02 06:33:03
38238,NaN,0027005,"Outside rubbish, trash or waste fire ...",2016/08/02,07,NaN,Elliot,AVE,S,,,,,,,0.0,0.0,151,2016/08/02,1,1,2100,True,2016-08-02 22:41:07,2016-08-02 22:56:13
38239,NaN,0027024,"Outside rubbish, trash or waste fire ...",2016/08/03,05,NaN,Cedar,AVE,S,,,E,Lake,ST,,0.0,0.0,151,2016/08/03,1,1,,True,2016-08-03 01:09:16,2016-08-03 01:19:09
38240,NaN,0027067,"Excessive heat, scorch burns with no ignition ...",2016/08/03,21,NaN,Lake,ST,E,,,,,,,0.0,0.0,251,2016/08/03,1,1,4222,True,2016-08-03 11:37:01,2016-08-03 14:39:18


In [39]:
years = sorted(fires['alarm_datetime'].dt.year.unique())
years

[2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020, 2021, 2022]

In [40]:
col_names = fires.columns
col_names = col_names.drop('number_', 'number').insert(5, 'number').drop_duplicates()
col_names
fires = fires[col_names]

fires.head()

,FID,inci_no,descript,alm_date,station,number,street,st_type,st_suffix,addr_2,apt_room,xst_prefix,xstreet,xst_type,xst_suffix,latitude,longitude,inci_type,clr_date,alarms,complete,confirmed_fire,alarm_datetime,clear_datetime
38236,NaN,0026882,Dumpster or other outside trash receptacle fir...,2016/08/02,21,2716,38,ST,E,,,,,,,0.0,0.0,154,2016/08/02,1,1,True,2016-08-02 05:44:50,2016-08-02 06:20:14
38237,NaN,0026886,Dumpster or other outside trash receptacle fir...,2016/08/02,06,,Franklin,AVE,W,,,S,Hennepin,AVE,,0.0,0.0,154,2016/08/02,1,1,True,2016-08-02 06:16:10,2016-08-02 06:33:03
38238,NaN,0027005,"Outside rubbish, trash or waste fire ...",2016/08/02,07,2100,Elliot,AVE,S,,,,,,,0.0,0.0,151,2016/08/02,1,1,True,2016-08-02 22:41:07,2016-08-02 22:56:13
38239,NaN,0027024,"Outside rubbish, trash or waste fire ...",2016/08/03,05,,Cedar,AVE,S,,,E,Lake,ST,,0.0,0.0,151,2016/08/03,1,1,True,2016-08-03 01:09:16,2016-08-03 01:19:09
38240,NaN,0027067,"Excessive heat, scorch burns with no ignition ...",2016/08/03,21,4222,Lake,ST,E,,,,,,,0.0,0.0,251,2016/08/03,1,1,True,2016-08-03 11:37:01,2016-08-03 14:39:18


In [41]:
df = fires.loc[fires.alarm_datetime.dt.year == 2021]

In [42]:
import matplotlib.pyplot as plt 
import numpy as np
import cartopy.crs as ccrs
import cartopy.io.img_tiles as cimgt
import io
from urllib.request import urlopen, Request
from PIL import Image


# This code uses a spoofing algorithm to avoid bounceback from OSM servers
#

def image_spoof(self, tile): # this function pretends not to be a Python script
    url = self._image_url(tile) # get the url of the street map API
    req = Request(url) # start request
    req.add_header('User-agent','Anaconda 3') # add user agent to request
    fh = urlopen(req) 
    im_data = io.BytesIO(fh.read()) # get image
    fh.close() # close url
    img = Image.open(im_data) # open image with PIL
    img = img.convert(self.desired_tile_form) # set image format
    return img, self.tileextent(tile), 'lower' # reformat for cartopy

In [43]:
def plot_fire_map(date_min, date_max):
    df = fires.loc[fires.alarm_datetime.dt.year == year]


    max_lat, min_lat = df.latitude.max(), df.latitude.min()
    max_lon, min_lon = df.longitude.max(), df.longitude.min()


    cimgt.OSM.get_image = image_spoof # reformat web request for street map spoofing
    osm_img = cimgt.OSM() # spoofed, downloaded street map


    fig = plt.figure(figsize=(12,15)) # open matplotlib figure
    ax = fig.add_subplot(1,1,1, projection=osm_img.crs) # project using coordinate reference system (CRS) of street map
    ax.set_global()
    zoom = 0.075 # for zooming out of center point
    extent = [min_lon, max_lon, min_lat, max_lat]
    ax.set_extent(extent) # set extents

    scale = np.ceil(-np.sqrt(2)*np.log(np.divide(zoom,350.0))) # empirical solve for scale based on zoom
    scale = (scale<20) and scale or 19 # scale cannot be larger than 19
    ax.add_image(osm_img, int(scale), zorder=0) # add OSM with zoom specification
    # NOTE: zoom specifications should be selected based on extent:
    # -- 2     = coarse image, select for worldwide or continental scales
    # -- 4-6   = medium coarseness, select for countries and larger states
    # -- 6-10  = medium fineness, select for smaller states, regions, and cities
    # -- 10-12 = fine image, select for city boundaries and zip codes
    # -- 14+   = extremely fine image, select for roads, blocks, buildings

    ax.scatter(df.longitude, df.latitude, alpha= 0.5, c='r', s=50,
                transform=ccrs.PlateCarree()) # NEED transform=crs.PlateCarree()

    plt.title('Minneapolis Fire Calls', size=24)
    plt.show() # show the plot

In [44]:
# plot_fire_map(2017)

In [45]:
# Years with some incorrect locations: 2016, 2020
fires.loc[fires.longitude > -90]

,FID,inci_no,descript,alm_date,station,number,street,st_type,st_suffix,addr_2,apt_room,xst_prefix,xstreet,xst_type,xst_suffix,latitude,longitude,inci_type,clr_date,alarms,complete,confirmed_fire,alarm_datetime,clear_datetime
38236,NaN,0026882,Dumpster or other outside trash receptacle fir...,2016/08/02,21,2716,38,ST,E,,,,,,,0.0,0.0,154,2016/08/02,1,1,True,2016-08-02 05:44:50,2016-08-02 06:20:14
38237,NaN,0026886,Dumpster or other outside trash receptacle fir...,2016/08/02,06,,Franklin,AVE,W,,,S,Hennepin,AVE,,0.0,0.0,154,2016/08/02,1,1,True,2016-08-02 06:16:10,2016-08-02 06:33:03
38238,NaN,0027005,"Outside rubbish, trash or waste fire ...",2016/08/02,07,2100,Elliot,AVE,S,,,,,,,0.0,0.0,151,2016/08/02,1,1,True,2016-08-02 22:41:07,2016-08-02 22:56:13
38239,NaN,0027024,"Outside rubbish, trash or waste fire ...",2016/08/03,05,,Cedar,AVE,S,,,E,Lake,ST,,0.0,0.0,151,2016/08/03,1,1,True,2016-08-03 01:09:16,2016-08-03 01:19:09
38240,NaN,0027067,"Excessive heat, scorch burns with no ignition ...",2016/08/03,21,4222,Lake,ST,E,,,,,,,0.0,0.0,251,2016/08/03,1,1,True,2016-08-03 11:37:01,2016-08-03 14:39:18
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
38296,NaN,0030463,Passenger vehicle fire ...,2016/08/28,16,1515,Washburn,AVE,N,,,,,,,0.0,0.0,131,2016/08/29,1,1,True,2016-08-28 22:53:05,2016-08-29 00:42:15
38297,NaN,0030615,Passenger vehicle fire ...,2016/08/30,06,1300-130,Interstate 94 Wb,,,,,,,,,0.0,0.0,131,2016/08/30,1,1,True,2016-08-30 01:08:49,2016-08-30 01:44:16
38298,NaN,0030851,"Outside rubbish, trash or waste fire ...",2016/08/31,05,3129,Oakland,AVE,,,,,,,,0.0,0.0,151,2016/08/31,1,1,True,2016-08-31 20:10:30,2016-08-31 20:18:47
38299,NaN,0030887,Passenger vehicle fire ...,2016/09/01,06,,14,ST,W,,,,Willow,ST,,0.0,0.0,131,2016/09/01,1,1,True,2016-09-01 06:24:02,2016-09-01 07:13:31


In [46]:
import requests
import urllib.parse

address = 'Shivaji Nagar, Bangalore, KA 560001'
url = 'https://nominatim.openstreetmap.org/search/' + urllib.parse.quote(address) +'?format=json'

response = requests.get(url).json()
print(response[0]["lat"])
print(response[0]["lon"])

12.9801559
77.6023714


In [47]:
fires.loc[fires.latitude == 0].head()

,FID,inci_no,descript,alm_date,station,number,street,st_type,st_suffix,addr_2,apt_room,xst_prefix,xstreet,xst_type,xst_suffix,latitude,longitude,inci_type,clr_date,alarms,complete,confirmed_fire,alarm_datetime,clear_datetime
38236,NaN,0026882,Dumpster or other outside trash receptacle fir...,2016/08/02,21,2716,38,ST,E,,,,,,,0.0,0.0,154,2016/08/02,1,1,True,2016-08-02 05:44:50,2016-08-02 06:20:14
38237,NaN,0026886,Dumpster or other outside trash receptacle fir...,2016/08/02,06,,Franklin,AVE,W,,,S,Hennepin,AVE,,0.0,0.0,154,2016/08/02,1,1,True,2016-08-02 06:16:10,2016-08-02 06:33:03
38238,NaN,0027005,"Outside rubbish, trash or waste fire ...",2016/08/02,07,2100,Elliot,AVE,S,,,,,,,0.0,0.0,151,2016/08/02,1,1,True,2016-08-02 22:41:07,2016-08-02 22:56:13
38239,NaN,0027024,"Outside rubbish, trash or waste fire ...",2016/08/03,05,,Cedar,AVE,S,,,E,Lake,ST,,0.0,0.0,151,2016/08/03,1,1,True,2016-08-03 01:09:16,2016-08-03 01:19:09
38240,NaN,0027067,"Excessive heat, scorch burns with no ignition ...",2016/08/03,21,4222,Lake,ST,E,,,,,,,0.0,0.0,251,2016/08/03,1,1,True,2016-08-03 11:37:01,2016-08-03 14:39:18


In [48]:
test_row = fires.loc[fires.latitude == 0].iloc[2]
address = re.sub('\s+', ' ', f'{test_row.number} {test_row.street} {test_row.st_type} {test_row.st_suffix.strip()}, Minneapolis, MN')
address

'2100 Elliot AVE S, Minneapolis, MN'

In [49]:
url = 'https://nominatim.openstreetmap.org/search/' + urllib.parse.quote(address) +'?format=json'

response = requests.get(url).json()
print(response[0]["lat"])
print(response[0]["lon"])

44.9617
-93.261202


In [50]:
def get_lat(row):
    if row.latitude == 0:
        address = re.sub('\s+', ' ', f'{row.number}+{row.street}+{row.st_type}+{row.st_suffix.strip()}+Minneapolis+MN')
        url = 'https://nominatim.openstreetmap.org/search/' + urllib.parse.quote(address) +'?format=json'

        response = requests.get(url).json()
        try:
            return response[0]["lat"]
        except:
            return np.nan
    else:
        return row.latitude

def get_lon(row):
    if row.longitude == 0:
        address = re.sub('\s+', ' ', f'{row.number}+{row.street}+{row.st_type}+{row.st_suffix.strip()}+Minneapolis+MN')
        url = 'https://nominatim.openstreetmap.org/search/' + urllib.parse.quote(address) +'?format=json'

        response = requests.get(url).json()
        try:
            return response[0]["lon"]
        except:
            return np.nan
    else:
        return row.longitude

In [51]:
fires.loc[fires.latitude == 0].head()

,FID,inci_no,descript,alm_date,station,number,street,st_type,st_suffix,addr_2,apt_room,xst_prefix,xstreet,xst_type,xst_suffix,latitude,longitude,inci_type,clr_date,alarms,complete,confirmed_fire,alarm_datetime,clear_datetime
38236,NaN,0026882,Dumpster or other outside trash receptacle fir...,2016/08/02,21,2716,38,ST,E,,,,,,,0.0,0.0,154,2016/08/02,1,1,True,2016-08-02 05:44:50,2016-08-02 06:20:14
38237,NaN,0026886,Dumpster or other outside trash receptacle fir...,2016/08/02,06,,Franklin,AVE,W,,,S,Hennepin,AVE,,0.0,0.0,154,2016/08/02,1,1,True,2016-08-02 06:16:10,2016-08-02 06:33:03
38238,NaN,0027005,"Outside rubbish, trash or waste fire ...",2016/08/02,07,2100,Elliot,AVE,S,,,,,,,0.0,0.0,151,2016/08/02,1,1,True,2016-08-02 22:41:07,2016-08-02 22:56:13
38239,NaN,0027024,"Outside rubbish, trash or waste fire ...",2016/08/03,05,,Cedar,AVE,S,,,E,Lake,ST,,0.0,0.0,151,2016/08/03,1,1,True,2016-08-03 01:09:16,2016-08-03 01:19:09
38240,NaN,0027067,"Excessive heat, scorch burns with no ignition ...",2016/08/03,21,4222,Lake,ST,E,,,,,,,0.0,0.0,251,2016/08/03,1,1,True,2016-08-03 11:37:01,2016-08-03 14:39:18


In [52]:
fires['new_lat'] = fires.apply(get_lat, axis=1)
fires['new_lon'] = fires.apply(get_lon, axis=1)

In [53]:
print(len(fires.loc[fires.latitude != fires.new_lat]))
print(len(fires.loc[fires.longitude != fires.new_lon]))

64
64


In [54]:
fires.head()

,FID,inci_no,descript,alm_date,station,number,street,st_type,st_suffix,addr_2,apt_room,xst_prefix,xstreet,xst_type,xst_suffix,latitude,longitude,inci_type,clr_date,alarms,complete,confirmed_fire,alarm_datetime,clear_datetime,new_lat,new_lon
38236,NaN,0026882,Dumpster or other outside trash receptacle fir...,2016/08/02,21,2716,38,ST,E,,,,,,,0.0,0.0,154,2016/08/02,1,1,True,2016-08-02 05:44:50,2016-08-02 06:20:14,NaN,NaN
38237,NaN,0026886,Dumpster or other outside trash receptacle fir...,2016/08/02,06,,Franklin,AVE,W,,,S,Hennepin,AVE,,0.0,0.0,154,2016/08/02,1,1,True,2016-08-02 06:16:10,2016-08-02 06:33:03,44.9627504,-93.3054853
38238,NaN,0027005,"Outside rubbish, trash or waste fire ...",2016/08/02,07,2100,Elliot,AVE,S,,,,,,,0.0,0.0,151,2016/08/02,1,1,True,2016-08-02 22:41:07,2016-08-02 22:56:13,44.9617,-93.261202
38239,NaN,0027024,"Outside rubbish, trash or waste fire ...",2016/08/03,05,,Cedar,AVE,S,,,E,Lake,ST,,0.0,0.0,151,2016/08/03,1,1,True,2016-08-03 01:09:16,2016-08-03 01:19:09,44.891355,-93.2468121
38240,NaN,0027067,"Excessive heat, scorch burns with no ignition ...",2016/08/03,21,4222,Lake,ST,E,,,,,,,0.0,0.0,251,2016/08/03,1,1,True,2016-08-03 11:37:01,2016-08-03 14:39:18,NaN,NaN


In [55]:
fires.loc[fires.new_lat.isna()]

,FID,inci_no,descript,alm_date,station,number,street,st_type,st_suffix,addr_2,apt_room,xst_prefix,xstreet,xst_type,xst_suffix,latitude,longitude,inci_type,clr_date,alarms,complete,confirmed_fire,alarm_datetime,clear_datetime,new_lat,new_lon
38236,NaN,0026882,Dumpster or other outside trash receptacle fir...,2016/08/02,21,2716,38,ST,E,,,,,,,0.0,0.0,154,2016/08/02,1,1,True,2016-08-02 05:44:50,2016-08-02 06:20:14,NaN,NaN
38240,NaN,0027067,"Excessive heat, scorch burns with no ignition ...",2016/08/03,21,4222,Lake,ST,E,,,,,,,0.0,0.0,251,2016/08/03,1,1,True,2016-08-03 11:37:01,2016-08-03 14:39:18,NaN,NaN
38244,NaN,0027398,"Natural vegetation fire, Other ...",2016/08/05,05,,24,AVE,S,,,E,32,ST,,0.0,0.0,140,2016/08/05,1,1,True,2016-08-05 18:35:03,2016-08-05 18:44:37,NaN,NaN
38248,NaN,0027619,Brush or brush-and-grass mixture fire ...,2016/08/07,04,1315,River,RD,N,,,,,,,0.0,0.0,142,2016/08/07,1,1,True,2016-08-07 12:22:14,2016-08-07 13:00:02,NaN,NaN
38249,NaN,0027623,Passenger vehicle fire ...,2016/08/07,15,,18,AVE,NE,,,NE,Lincoln,ST,,0.0,0.0,131,2016/08/07,1,1,True,2016-08-07 13:32:44,2016-08-07 13:45:29,NaN,NaN
38254,NaN,0027945,Brush or brush-and-grass mixture fire ...,2016/08/10,04,1401,River,RD,N,,,,,,,0.0,0.0,142,2016/08/10,1,1,True,2016-08-10 00:46:49,2016-08-10 01:31:25,NaN,NaN
38256,NaN,0027961,"Outside rubbish, trash or waste fire ...",2016/08/10,08,304,Lake,ST,E,,,,,,,0.0,0.0,151,2016/08/10,1,1,True,2016-08-10 05:31:14,2016-08-10 05:51:53,NaN,NaN
38257,NaN,0028044,Dumpster or other outside trash receptacle fir...,2016/08/10,14,200-201,Interstate 94 Wb,,,,,,,,,0.0,0.0,154,2016/08/10,1,1,True,2016-08-10 17:35:21,2016-08-10 18:14:19,NaN,NaN
38265,NaN,0028438,Passenger vehicle fire ...,2016/08/13,04,525,7,ST,N,,,,,,,0.0,0.0,131,2016/08/13,1,1,True,2016-08-13 10:04:18,2016-08-13 10:33:06,NaN,NaN
38273,NaN,0028946,Passenger vehicle fire ...,2016/08/17,06,100,Interstate 94 Eb,,,,,,,,,0.0,0.0,131,2016/08/17,1,1,True,2016-08-17 09:46:12,2016-08-17 10:47:25,NaN,NaN


In [56]:
fires.loc[fires.new_lat.isna()]

,FID,inci_no,descript,alm_date,station,number,street,st_type,st_suffix,addr_2,apt_room,xst_prefix,xstreet,xst_type,xst_suffix,latitude,longitude,inci_type,clr_date,alarms,complete,confirmed_fire,alarm_datetime,clear_datetime,new_lat,new_lon
38236,NaN,0026882,Dumpster or other outside trash receptacle fir...,2016/08/02,21,2716,38,ST,E,,,,,,,0.0,0.0,154,2016/08/02,1,1,True,2016-08-02 05:44:50,2016-08-02 06:20:14,NaN,NaN
38240,NaN,0027067,"Excessive heat, scorch burns with no ignition ...",2016/08/03,21,4222,Lake,ST,E,,,,,,,0.0,0.0,251,2016/08/03,1,1,True,2016-08-03 11:37:01,2016-08-03 14:39:18,NaN,NaN
38244,NaN,0027398,"Natural vegetation fire, Other ...",2016/08/05,05,,24,AVE,S,,,E,32,ST,,0.0,0.0,140,2016/08/05,1,1,True,2016-08-05 18:35:03,2016-08-05 18:44:37,NaN,NaN
38248,NaN,0027619,Brush or brush-and-grass mixture fire ...,2016/08/07,04,1315,River,RD,N,,,,,,,0.0,0.0,142,2016/08/07,1,1,True,2016-08-07 12:22:14,2016-08-07 13:00:02,NaN,NaN
38249,NaN,0027623,Passenger vehicle fire ...,2016/08/07,15,,18,AVE,NE,,,NE,Lincoln,ST,,0.0,0.0,131,2016/08/07,1,1,True,2016-08-07 13:32:44,2016-08-07 13:45:29,NaN,NaN
38254,NaN,0027945,Brush or brush-and-grass mixture fire ...,2016/08/10,04,1401,River,RD,N,,,,,,,0.0,0.0,142,2016/08/10,1,1,True,2016-08-10 00:46:49,2016-08-10 01:31:25,NaN,NaN
38256,NaN,0027961,"Outside rubbish, trash or waste fire ...",2016/08/10,08,304,Lake,ST,E,,,,,,,0.0,0.0,151,2016/08/10,1,1,True,2016-08-10 05:31:14,2016-08-10 05:51:53,NaN,NaN
38257,NaN,0028044,Dumpster or other outside trash receptacle fir...,2016/08/10,14,200-201,Interstate 94 Wb,,,,,,,,,0.0,0.0,154,2016/08/10,1,1,True,2016-08-10 17:35:21,2016-08-10 18:14:19,NaN,NaN
38265,NaN,0028438,Passenger vehicle fire ...,2016/08/13,04,525,7,ST,N,,,,,,,0.0,0.0,131,2016/08/13,1,1,True,2016-08-13 10:04:18,2016-08-13 10:33:06,NaN,NaN
38273,NaN,0028946,Passenger vehicle fire ...,2016/08/17,06,100,Interstate 94 Eb,,,,,,,,,0.0,0.0,131,2016/08/17,1,1,True,2016-08-17 09:46:12,2016-08-17 10:47:25,NaN,NaN


In [57]:
fires.drop(['latitude', 'longitude'], axis=1).rename(columns={'new_lat':'latitude', 'new_lon':'longitude'}).to_csv('../data/confirmed_fires.csv', index=False)

In [58]:
min_date = date(2020, 1, 1)
max_date = date(2020, 12, 31)
# df.loc[min_date <= df.alarm_datetime.dt.date <= max_date]

In [59]:
min_date

datetime.date(2020, 1, 1)

In [60]:
max_date

datetime.date(2020, 12, 31)

In [61]:
fires.alarm_datetime

38236   2016-08-02 05:44:50
38237   2016-08-02 06:16:10
38238   2016-08-02 22:41:07
38239   2016-08-03 01:09:16
38240   2016-08-03 11:37:01
                ...        
46033   2022-04-18 10:31:57
46245   2022-04-19 09:02:55
46246   2022-04-19 11:49:26
46247   2022-04-19 12:31:35
46303   2022-04-20 10:21:29
Name: alarm_datetime, Length: 49525, dtype: datetime64[ns]

In [62]:
df = fires
year_choice = 2020
fires.loc[fires.alarm_datetime.dt.year == year_choice].alarm_datetime.min().date()

datetime.date(2020, 1, 1)

In [63]:
# fires.alarm_datetime.dt.date.apply(date.strftime('M D, YYYY'))

In [64]:
max_date.strftime('%B %d, %Y')

'December 31, 2020'

# Working on map areas

In [68]:
districts = gpd.read_file('../data/Fire_Districts.geojson')
districts = districts.drop(['SHAPE_Length', 'SHAPE_Area'], axis=1).rename(columns={'DISTRICT':'District'})
districts

,FID,District,geometry
0,1,1,"MULTIPOLYGON (((-93.26367 44.98413, -93.26312 ..."
1,2,2,"MULTIPOLYGON (((-93.30455 44.96643, -93.30375 ..."
2,3,3,"MULTIPOLYGON (((-93.24455 44.97807, -93.24436 ..."
3,4,4,"MULTIPOLYGON (((-93.31950 45.05125, -93.31741 ..."
4,5,5,"MULTIPOLYGON (((-93.25916 44.98427, -93.25934 ..."
5,6,5,"MULTIPOLYGON (((-93.27726 45.03657, -93.27726 ..."


In [69]:
districts.explore('District', cmap='Dark2')

AttributeError: 'GeoDataFrame' object has no attribute 'explore'

In [70]:
station_areas = gpd.read_file('../data/Fire_Stations_Areas.geojson')
station_areas = station_areas.rename(columns={'STATION':'Station'})[['Station', 'geometry']]
station_areas

,Station,geometry
0,1,"POLYGON ((-93.26367 44.98413, -93.26312 44.983..."
1,11,"POLYGON ((-93.25916 44.98427, -93.25934 44.984..."
2,11,"POLYGON ((-93.24532 45.00237, -93.24499 45.002..."
3,12,"POLYGON ((-93.23324 44.92330, -93.23319 44.923..."
4,14,"POLYGON ((-93.31483 45.02952, -93.31355 45.029..."
5,15,"POLYGON ((-93.22690 45.02773, -93.22688 45.025..."
6,16,"POLYGON ((-93.30444 44.99742, -93.30444 44.996..."
7,17,"POLYGON ((-93.26647 44.94667, -93.26645 44.945..."
8,19,"POLYGON ((-93.21516 44.98977, -93.21520 44.987..."
9,2,"POLYGON ((-93.27726 45.03657, -93.27726 45.035..."


In [71]:
station_areas.explore('Station')

AttributeError: 'GeoDataFrame' object has no attribute 'explore'

In [72]:
station_areas

,Station,geometry
0,1,"POLYGON ((-93.26367 44.98413, -93.26312 44.983..."
1,11,"POLYGON ((-93.25916 44.98427, -93.25934 44.984..."
2,11,"POLYGON ((-93.24532 45.00237, -93.24499 45.002..."
3,12,"POLYGON ((-93.23324 44.92330, -93.23319 44.923..."
4,14,"POLYGON ((-93.31483 45.02952, -93.31355 45.029..."
5,15,"POLYGON ((-93.22690 45.02773, -93.22688 45.025..."
6,16,"POLYGON ((-93.30444 44.99742, -93.30444 44.996..."
7,17,"POLYGON ((-93.26647 44.94667, -93.26645 44.945..."
8,19,"POLYGON ((-93.21516 44.98977, -93.21520 44.987..."
9,2,"POLYGON ((-93.27726 45.03657, -93.27726 45.035..."


In [73]:
districts

,FID,District,geometry
0,1,1,"MULTIPOLYGON (((-93.26367 44.98413, -93.26312 ..."
1,2,2,"MULTIPOLYGON (((-93.30455 44.96643, -93.30375 ..."
2,3,3,"MULTIPOLYGON (((-93.24455 44.97807, -93.24436 ..."
3,4,4,"MULTIPOLYGON (((-93.31950 45.05125, -93.31741 ..."
4,5,5,"MULTIPOLYGON (((-93.25916 44.98427, -93.25934 ..."
5,6,5,"MULTIPOLYGON (((-93.27726 45.03657, -93.27726 ..."


In [74]:
gpd.overlay(station_areas, districts).explore('District', cmap='Dark2')
# station_areas.explore()

AttributeError: 'GeoDataFrame' object has no attribute 'explore'

In [ ]:
nhoods = gpd.read_file('../data/Minneapolis_Neighborhoods.geojson')
nhoods = nhoods.rename(columns={'SYMBOL_NAM':'symbol_name', 'BDNAME':'nhood', 'BDNUM':'BDnum'})[['symbol_name', 'nhood', 'BDnum', 'geometry']]
nhoods

In [ ]:
station_districts = gpd.overlay(station_areas, districts)
station_districts.head()

In [ ]:
nhood_districts = gpd.overlay(nhoods, station_districts, keep_geom_type=False)
nhood_districts.head()

In [ ]:
nhood_districts.explore('District',
     cmap='Dark2',
     tooltip = ['nhood', 'Station', 'District'])

In [ ]:
confirmed_fires = pd.read_csv('../data/confirmed_fires.csv')

In [ ]:
fires_gdf = gpd.GeoDataFrame(confirmed_fires, geometry=gpd.points_from_xy(confirmed_fires.longitude, confirmed_fires.latitude), crs='EPSG:4326')
fires_gdf.head()

In [ ]:
fires_gdf.drop(['latitude', 'longitude', 'number', 'street', 'st_type', 'st_suffix', 'addr_2', 'apt_room', 'xst_prefix', 'xstreet', 'xst_type', 'xst_suffix'], 
    axis=1, inplace=True)

In [ ]:
fires_gdf.head()

In [ ]:
fires_gdf.info()

In [ ]:
nhood_districts

In [ ]:
districts

In [ ]:
fires_gdf[['alarm_datetime', 'clear_datetime', 'geometry']]

In [ ]:
def filter_fires(df, min_date, max_date):
    df = df.rename(columns={'station':'station_from_fires'})[['station_from_fires', 'alarm_datetime', 'clear_datetime', 'geometry']]
    df['alarm_datetime'] = pd.to_datetime(df['alarm_datetime'])
    df['clear_datetime'] = pd.to_datetime(df['clear_datetime'])

    date_mask = (min_date <= fires_gdf.alarm_datetime.dt.date) & (fires_gdf.alarm_datetime.dt.date <= max_date)

    return df.loc[date_mask][['station_from_fires', 'geometry']]

In [ ]:
filter_fires(fires_gdf, date(2012, 1, 1), date(2022, 4, 22))

In [ ]:
filtered_fires = filter_fires(fires_gdf, date(2012, 1, 1), date(2022, 4, 22))

gpd.sjoin(districts, filtered_fires).groupby('District')['District'].count()

In [ ]:
def get_counts(filtered_fires, areas, area_type='District'):
    gdf = gpd.sjoin(areas, filtered_fires)

    counts = gdf.groupby(area_type)[area_type].count()
    counts = pd.DataFrame(counts).rename(columns={area_type:'num_of_fires'}).reset_index()
    return counts

In [ ]:
districts.merge(get_counts(filtered_fires, districts), on='District')

In [ ]:
map_district_fire_counts(filtered_fires, districts)

In [ ]:
filtered_fires = filter_fires(fires_gdf, date(2012, 1, 1), date(2022, 4, 22))

gpd.sjoin(districts, filtered_fires)

In [ ]:
areas_with_counts.info()

In [ ]:
import folium

In [ ]:
def map_fire_counts(filtered_fires, areas, area_type='District'):
    areas_with_counts = areas.merge(get_counts(filtered_fires, areas, area_type), on=area_type)

    if area_type == 'District':
        tooltip_fields = ['District', 'num_of_fires']
        tooltip_aliases = ['District:', 'Number of Fires:']
    elif area_type == 'Station':
        tooltip_fields = ['Station', 'District', 'num_of_fires']
        tooltip_aliases = ['Station:', 'District:', 'Number of Fires:']
    elif area_type == 'nhood':
        tooltip_fields = ['nhood', 'Station', 'District', 'num_of_fires']
        tooltip_aliases = ['Neighborhood:', 'Station:', 'District:', 'Number of Fires:']


    m = folium.Map(location=[44.986656, -93.258133], zoom_start=12)

    folium.Choropleth(
        geo_data = areas_with_counts,
        data = areas_with_counts,
        columns=[area_type, 'num_of_fires'],
        key_on=f'feature.properties.{area_type}',
        fill_color='YlOrRd'
        ).add_to(m)

    areas = folium.GeoJson(
        areas_with_counts,
        style_function = lambda features: {
            'fillOpacity': 0,
            'weight': 0
        }
        )
    areas.add_child(
        folium.features.GeoJsonTooltip(
            fields = tooltip_fields,
            aliases = tooltip_aliases
        )
    )

    areas.add_to(m)

    c = plt.colorbar()
    plt.clim(0, areas_with_counts['num_of_fires'].max())

    return m

In [ ]:
map_fire_counts(filtered_fires, districts, 'District')

In [ ]:
map_fire_counts(filtered_fires, station_districts, 'Station')

In [ ]:
def fix_geometry_collection(geom):
    if geom.geom_type == 'GeometryCollection':
        multi_list = []

        for thing in geom:
            if thing.geom_type == 'Polygon':
                multi_list.append(thing)

        return MultiPolygon(multi_list)
    else:
        return geom

In [ ]:
nhood_districts['geometry'] = nhood_districts.geometry.apply(fix_geometry_collection)

In [ ]:
map_fire_counts(filtered_fires, nhood_districts, 'nhood')